In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import argrelextrema
from tqdm.notebook import tqdm
import mplfinance as mpf

import sys

sys.path.insert(0,'/Users/orentapiero/btc_research/')

from btc_functions.glassnode import *
from btc_functions.import_data import get_glassnode_price,get_glassnode_data
from btc_functions.variable_list_urls import *
from btc_functions.utilities import strided_app,strided_app2

plt.rcParams['figure.figsize'] = [18, 10]
sns.set()

Urls['transfers_volume_sum'] = 'https://api.glassnode.com/v1/metrics/transactions/transfers_volume_sum'


In [3]:
ohlc = get_glassnode_price()
ohlc = ohlc.rename(columns = {'o':'Open','h':'High','l':'Low','c':'Close'})
ohlc = ohlc

selected = ['marketcap_usd',
            'mvrv_z_score',
            'sopr_adjusted',
            'puell_multiple',
            'net_unrealized_profit_loss',
            'transfers_volume_sum',
            'transfers_volume_exchanges_net',
            'dormancy_flow',
            'reserve_risk',
            'cdd90_age_adjusted',
            'average_dormancy',
            'liveliness', 
            'realized_profits_to_value_ratio',
            'rhodl_ratio',
            'cvdd',
            'nvts', 
            'marketcap_thermocap_ratio',
            'difficulty_latest', 
            'non_zero_count']

features = get_glassnode_data(selected,Urls)
features = features#.loc[:'2019']


  0%|          | 0/19 [00:00<?, ?it/s]

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

def create_target(ohlc,L = 30):
    px = np.log(ohlc.Close)
    target_df = pd.DataFrame(px)
    target_df['R'] = px.diff()
    target_df['RV'] = np.sqrt((target_df['R']**2).rolling(L).sum())
    target_df['FR'] = (px.shift(-L) - px)
    target_df['FR2RV'] = target_df['FR']/target_df['RV']
    target_df['Target'] = 0.
    target_df.loc[target_df.FR2RV>=1.,'Target']=1.
    target_df.loc[target_df.FR2RV<=-1.,'Target']=-1.
    return target_df



In [15]:
create_target(ohlc,L = 30)

,Close,R,RV,FR,FR2RV,Target
t,,,,,,
2010-07-17,-3.005581,NaN,NaN,0.279875,NaN,0.0
2010-07-18,-2.455270,0.550310,NaN,-0.203990,NaN,0.0
2010-07-19,-2.515778,-0.060508,NaN,-0.172469,NaN,0.0
2010-07-20,-2.593829,-0.078051,NaN,-0.113721,NaN,0.0
2010-07-21,-2.535653,0.058176,NaN,-0.190052,NaN,0.0
...,...,...,...,...,...,...
2022-03-10,10.582922,-0.061964,0.237285,NaN,NaN,0.0
2022-03-11,10.566472,-0.016450,0.237794,NaN,NaN,0.0
2022-03-12,10.569647,0.003176,0.237119,NaN,NaN,0.0
